In [3]:
pip install python-dotenv

  Using cached python_dotenv-0.19.1-py2.py3-none-any.whl (17 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install ndjson

Note: you may need to restart the kernel to use updated packages.


In [103]:
pip install demoji

  Using cached demoji-1.1.0-py3-none-any.whl (42 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import smtplib
import os
import json
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
from email.utils import formatdate
from os.path import join, dirname
import time

msg = MIMEMultipart()

import pandas as pd
import numpy as np
import ndjson
from dotenv import load_dotenv
import sentence_vectorizer
import functools
import demoji
from ja_sentence_segmenter.common.pipeline import make_pipeline
from ja_sentence_segmenter.concatenate.simple_concatenator import concatenate_matching
from ja_sentence_segmenter.normalize.neologd_normalizer import normalize
from ja_sentence_segmenter.split.simple_splitter import split_newline, split_punctuation

import liking_users
import liked_tweets
import sentence_vectorizer

# dotenv_path = join(dirname(__file__), '.env')
# load_dotenv(dotenv_path)

NameError: name '__file__' is not defined

In [3]:
class Client():

    DIR = 'data/mail_body.txt'

    def __init__(self, client_data):

        self.client_data = client_data
        self.execution_time = 0
        self.target_user = []
        self.liked_tweet = []
        self.degree = []

    def extract_data(self):
        """クライアントの辞書から値をそれぞれ抽出
        """
        # 送信先を抽出    
        self.mail_address = self.client_data['mail_address']
        # 宛名を抽出
        self.name = self.client_data['name']
        # キーワードを抽出
        self.keyword = self.client_data['keyword']
        # ツイッターIDを抽出
        self.target_tweet_id = self.client_data['twitter_id']
        # ツイッター本文を抽出
        df_tweets = pd.DataFrame(self.client_data['tweets'])
        self.target_tweet = df_tweets.iloc[:,1][df_tweets.iloc[:,2]==self.target_tweet_id].values[0]

    def send_email(self):
        """クライアント宛にメール送信
        """
        send_address = os.environ.get('MAIL_ADDRESS')
        password = os.environ.get('MAIL_PASSWORD')

        with open(DIR, encoding = "utf-8") as f:
            body_temp = f.read()

        subject = 'Twitter分析結果'
        from_address = send_address
        to_address = self.mail_address

        # SMTPサーバに接続
        smtpobj = smtplib.SMTP('smtp.gmail.com', 587)
        smtpobj.starttls()
        smtpobj.login(send_address, password)

        # メール作成
        msg = MIMEMultipart()
        msg['Subject'] = subject
        msg['From'] = from_address
        msg['To'] = to_address
        msg['Date'] = formatdate()

        # メール本文
        # 一番大きい類似度のインデックス
        max_idx = self.degree.index(max(self.degree))
        
        body_text = body_temp.format(name=self.name,\
            keyword=self.keyword, target_tweet=self.target_tweet,\
            target_user=self.target_user[max_idx], liked_tweet=self.liked_tweet[max_idx],\
            degree=max(self.degree), execution_time=self.execution_time)
        body = MIMEText(body_text)
        msg.attach(body)

        # 作成したメールを送信
        smtpobj.send_message(msg)
        smtpobj.close()    

In [9]:
DIR = 'data/db_tweets.ndjson'

def main(dir):
    """メイン
    """
    client_data_list = []

    with open(dir, 'r+') as json_file:
        # 現在DBにあるクライアント情報を取得
        json_client_data = ndjson.load(json_file)
        # DBの内容を空にする
        # json_file.truncate(0) 

        for data in json_client_data:
            # 開始時間
            start_time = time.perf_counter()
            # クライアントクラスの初期化
            client = Client(data)
            # クライアントデータ準備
            client.extract_data()
            
            df_target_segments = segment_target_tweet(client.target_tweet)
            df_target_segments = vectorize_target_segments(df_target_segments)

            df_users = get_liking_users(client.target_tweet_id)
            df_liked_tweets = get_liking_users_liked_tweets(df_users, client.target_tweet)
            candidate_users = select_tweets(client.keyword, df_liked_tweets)
            candidate_users_segments = segment_tweet(candidate_users)
            df_candidate_users = vectorize_candidate_segments(candidate_users_segments)
            #dic_degree, dic_index = calculate_cosine_similarity(df_target_segments, df_candidate_users)
            derived_result(client, df_target_segments, df_candidate_users, df_users, df_liked_tweets)
            
            client.send_email()
            # 終了時間
            execution_time = time.perf_counter() - start_time
            client.execution_time = execution_time

    return client
                 
client = main(DIR)


Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


403


Exception: Request returned an error: 403 {"client_id":"22371308","detail":"When authenticating requests to the Twitter API v2 endpoints, you must use keys and tokens from a Twitter developer App that is attached to a Project. You can create a project via the developer portal.","registration_url":"https://developer.twitter.com/en/docs/projects/overview","title":"Client Forbidden","required_enrollment":"Standard Basic","reason":"client-not-enrolled","type":"https://api.twitter.com/2/problems/client-forbidden"}

In [307]:
print(client.execution_time)

42.75378513800388


In [308]:
client.keyword

'#udemy'

In [309]:
maxidx = client.degree.index(max(client.degree))
print(f'類似度スコア：{max(client.degree)}')
print()
print(client.liked_tweet[maxidx]) 

類似度スコア：0.901680052280426

#今日の積み上げ 11/9

#udemy ウェブ開発入門完全攻略コース
JavaScript入門　32%終了

講座を変えて、4度目のJS挑戦

昨日、自社システムのトップページを作りましたが
このページがページで終わらず、機能を実装させて、必ず生産性として役立てます💪

#駆け出しエンジニア #プログラミング初心者 https://t.co/Do7aQ2cn6Q


In [ ]:
for degree_lst, index_lst in zip(dic_degree.values(), dic_index.values()):
        
    client.degree.append(max(degree_lst))
    max_idx = degree_lst.index(max(degree_lst))
    target_user_id = df_candidate_users.iloc[max_idx].name
    # ターゲットユーザー名
    print(df_users['username'][df_users['id']==target_user_id].values)
    client.target_user.append(df_users['username'][df_users['id']==target_user_id])
    # ツイート断片を抽出
    segment_index = index_lst[max_idx]
    segment = df_candidate_users.loc[target_user_id,'text'][segment_index]
    # ツイート断片を含むツイート本文を抽出
    df_mask = df_liked_tweets[target_user_id].str.contains(segment)
    target_row = df_mask.first_valid_index()
    # print(target_row)
    # print(df_liked_tweets[target_user_id].iloc[target_row])
    client.liked_tweet.append(df_liked_tweets[target_user_id].iloc[target_row])

    print(client.degree)
    print(client.target_user)
    print(client.liked_tweet)

In [5]:
def get_liking_users(target_tweet_id):
    """ターゲットツイートにいいねしたユーザー情報をデータフレームで返す
    """
    # ターゲットツイートIDからいいねしたユーザーを検索
    json_response = liking_users.get_users(target_tweet_id)
    # いいねしたユーザー情報のデータフレーム
    return pd.json_normalize(json_response['data'])

def get_liking_users_liked_tweets(df_users, target_tweet):
    """ユーザーデータフレームからユーザーがいいねしたツイートのデータフレームを返す
    """    
    # ユーザーIDからいいねしたツイートを検索
    dic_liked_tweets = liked_tweets.get_liked_tweets(df_users['id'])
    # ユーザーごとの辞書データを結合したマルチカラムデータフレームを作成
    df_liked_tweets = pd.concat({k: pd.DataFrame(v['data']) for k, v in dic_liked_tweets.items()}, \
        axis=0).unstack(0).swaplevel(1,0, axis=1).sort_index(axis=1)
    # ユーザーIDを抽出    
    user_ids = df_liked_tweets.columns.get_level_values(0).unique()

    tweets_text_list = []
    for ids in user_ids:
        user_texts = df_liked_tweets[ids]['text'].copy()
        # いいねしたツイートにターゲットツイートが含まれていたらNANに置換する
        user_texts.replace(target_tweet, np.nan, inplace=True)
        tweets_text_list.append(user_texts)
        # いいねしたツイートのデータフレーム
    df_tweets_text = pd.DataFrame(tweets_text_list, index=user_ids)
    return df_tweets_text.T     


In [6]:
# ユーザーのいいねツイートへの前処理用

def select_tweets(keyword, df_liked_tweets):
    """ツイートのデータフレームからキーワードを含む文章をもつユーザーシリーズを作成
    """  
    for col in df_liked_tweets.columns:
        flag = df_liked_tweets[col].str.contains(keyword, na=False)
        df_liked_tweets.loc[~flag, col] = np.nan

    # NAN以外の値を持つユーザーIDを特定する（POSTする候補）
    exist_flag = df_liked_tweets.count() != 0
    user_name = df_liked_tweets.T[exist_flag].index

    # POSTするユーザー候補ごとに関連ツイート文を全て結合
    candidate_users = df_liked_tweets[user_name].T
    # 後で文を区切る区切りのためツイート文の文末に'。'を追加
    candidate_users += '。'
    # ツイート文を結合
    return candidate_users.iloc[:,0].str.cat([candidate_users.iloc[:,col] for col in candidate_users.columns[1:]], na_rep='')

def segment_tweet(candidate_users):
    """ツイートを文単位でセパレートしたユーザーシリーズを返す
    """  
    split_punc2 = functools.partial(split_punctuation, punctuations=r"。!?")
    concat_tail_no = functools.partial(concatenate_matching, former_matching_rule=r"^(?P<result>.+)(の)$", remove_former_matched=False)
    segmenter = make_pipeline(normalize, split_newline, concat_tail_no, split_punc2)

    sentence_list = []
    for text in candidate_users:
        sentence_list.append(list(segmenter(text)))

    candidate_users_segments = pd.Series(sentence_list, index=candidate_users.index, name='text')

    for i in range(len(candidate_users_segments)):
        candidate_users_segments[i] = [demoji.replace(string=seg, repl='') for seg in candidate_users_segments[i] if '。' != seg]

    return candidate_users_segments

def vectorize_candidate_segments(candidate_users_segments):
    """ユーザーシリーズに文単位のベクトルを追加したデータフレームを返す
    """  
    # ベクトル表現を生成する
    BSV = sentence_vectorizer.BertSequenceVectorizer()
    vectors_list = [pd.Series(text).map(lambda x: BSV.vectorize(x)) for text in candidate_users_segments]
    sr_vectors = pd.Series(vectors_list, index=candidate_users_segments.index, name='vector')
    return pd.concat([candidate_users_segments, sr_vectors], axis=1)



In [7]:
# ターゲットツイートへの前処理

def segment_target_tweet(target_tweet):
    """ターゲットツイートを文単位でセパレートしたデータフレームを返す
    """  
    split_punc2 = functools.partial(split_punctuation, punctuations=r"。!?")
    concat_tail_no = functools.partial(concatenate_matching, former_matching_rule=r"^(?P<result>.+)(の)$", remove_former_matched=False)
    segmenter = make_pipeline(normalize, split_newline, concat_tail_no, split_punc2)

    return pd.DataFrame(segmenter(target_tweet), columns=['text'])

def vectorize_target_segments(df_target_segments):
    """文単位のターゲットツイートデータフレームにベクトルを追加
    """  
    BSV = sentence_vectorizer.BertSequenceVectorizer()
    df_target_segments['vector'] = df_target_segments['text'].map(lambda x: BSV.vectorize(x))
    return df_target_segments

In [8]:
def calculate_cosine_similarity(df_target_segments, df_candidate_users):
    """ターゲットツイートとターゲットユーザーツイートの類似度を計算
    """     

    # コサイン類似度の高いユーザーを算出
    dic_degree = {}
    dic_index = {}

    for vec in df_candidate_users['vector']:

        join = df_target_segments['vector'].append(vec)
        for idx in range(len(df_target_segments)):
            # ターゲットツイートセグメントのインデックスを選択するマスク
            mask = np.ones(len(join), dtype=bool)
            mask[:len(df_target_segments)+1] = False
            mask[idx] = True
            # 類似度行列
            matrix = sentence_vectorizer.cos_sim_matrix(np.stack(join[mask]))
            #print(np.sort(matrix[0])[::-1])
            # ユーザーごとに最大の類似度とそのターゲットユーザーツイートのセグメントインデックスを取得
            degree = np.sort(matrix[0])[::-1][1]
            index = np.argsort(matrix[0])[::-1][1]

            dic_degree.setdefault(idx, []).append(degree)
            dic_index.setdefault(idx, []).append(index)

    return dic_degree, dic_index

def derived_result(client, df_target_segments, df_candidate_users, df_users, df_liked_tweets):
    """メール送信するユーザー名、ユーザーがいいねした関連度の高いツイート、関連スコア（0〜1）を返す
    """
    dic_degree, dic_index = calculate_cosine_similarity(df_target_segments, df_candidate_users)
    for degree_lst, index_lst in zip(dic_degree.values(), dic_index.values()):
        # 最大の関連度スコア
        client.degree.append(max(degree_lst))
        max_idx = degree_lst.index(max(degree_lst))
        target_user_id = df_candidate_users.iloc[max_idx].name
        # ターゲットユーザー名
        client.target_user.append(df_users['username'][df_users['id']==target_user_id].values)
        segment_index = index_lst[max_idx]
        # ツイート断片を抽出
        segment = df_candidate_users.loc[target_user_id,'text'][segment_index]
        # ツイート断片を含むツイート本文を抽出
        df_mask = df_liked_tweets[target_user_id].str.contains(segment)
        target_row = df_mask.first_valid_index()
        client.liked_tweet.append(df_liked_tweets.loc[target_row, target_user_id])

    print(client.degree, client.target_user, client.liked_tweet)

In [ ]:
        self.target_user = []
        self.liked_tweet = []
        self.degree = []

In [97]:
# このセルのdf_liked_tweets変数の中身はキャッシュで残ってるもの
# リスタートしたら消える

user_ids = df_liked_tweets.columns.get_level_values(0).unique()

tweets_text_list = []
for ids in user_ids:
    user_texts = df_liked_tweets[ids]['text'].copy()
    # いいねしたツイートにターゲットツイートが含まれていたらNANに置換する
    user_texts.replace(twlist[1], np.nan, inplace=True)
    tweets_text_list.append(user_texts)
    # いいねしたツイートのデータフレーム
df_tweets_text = pd.DataFrame(tweets_text_list, index=user_ids).T
df_tweets_text.head(3)

,1175285742001475586,1260973997845393408,1279267537629278211,1282640602396364802,1290624958603829248,1297906406155104256,1356973982,1414520142,1415640388241555461,1449028461310332936,963753537053143042
0,まだ新規申し込みは停止中。\n再会時には情報共有します。\n\n#駆け出しエンジニアと繋がり...,「マッチングアプリ」合格できました！ https://t.co/13jNVhiZmp #Re...,AR/VRには大きなポテンシャルがあると思いますが、Oculus Questに特化したコンテ...,私と迫さん（@yuki_99_s）のYouTube裏ノウハウBrainですが、3カ月以上経っ...,2021/11/08 8日目 作業時間:1h30m\n\nレスポンシブDog3まで完成！\n...,【#tomoya_yoshimoto_in_isub #isub】from slack\n快...,逆運動学に悩む・・・\nhttps://t.co/U1vksBjNQi\n\n#Arduin...,離職率の高い会社ってすごいチャンスだったりする。\n\n先輩や上司がどんどん辞めていくので、...,#今日の積み上げ\n今日はエナジードリンクの缶を積みました‼️ フリーランスとして独立してい...,昨日今日と停電&amp;ネット絶不調でプログラミング学習はおろか、SNSさえまともにできない...,"Rails Modelで整数値に限定したけば\nvalidates :user_id, nu..."
1,@banburero001 嘘というか、商材屋が人を騙すためのエサですね。\n月収○桁！！と...,服装に自信がない #プログラマー 必見！\n#エンジニア 向けブランド！\n\n#プログラミ...,#ギークハウス 品川・五反田・大崎では住人を募集中!\n\nコンセプトはエンジニア同士(志望...,おはようございます☀️\n\n「誰でもできること」を「誰もできないくらい続ける」が成功のコツ...,#デイトラ\n分からないところガンガン呟いていきます😵‍💫\n\n中級編でレスポンシブの\n...,自分はみんなを助けたいと言っても、結局誰も人が助けを求めて来ないのならそれはまずその人に人を...,頭悪い奴は #ゲーム だろうが攻略法見てりゃ良い\n\n#プログラミング など専門知識にして...,持つべきは絶対的な危機感\n会社員時代には終身雇用で安泰と思わないこと。常に疑問符をもち10...,[From ﾕﾃﾞﾀﾏｸﾞｫbot]\n🐣 #今日の積み上げ\n#駆け出しエンジニアと繋がり...,Progateでプログラミング勉強始めました！！\nHTML＆CSS 初級編やってます！\n...,🙇🏻‍♂️御礼🙇🏻‍♂️\n\nクラウドワークスで何度応募してもダメな人のためのBrain（...
2,かごもく #50 認証デバイス特集 (FIDO2 とか Yubikey とか) / http...,N予備校のプログラミング入門コースはゴールが『Webエンジニアとして内定もらって働ける』レベ...,【2021年最新版】\n 国内最大手のプログラミングスクール\n侍エンジニア塾の気になる料金...,色々と記事にして共有していきたい知識がここ1年でたくさん身についた気がします！\n\nいま行...,実戦した中にかなり効果的な勉強は\nゴールから始める事です！\n資格なら過去問から解く\n制...,トリニタスお仕事情報♪高額案件あり♪スキルアップ♪選べる案件♪ https://t.co/t...,2021-11-08 17:00:02\n仙台市\n現在の天候：所により曇り\n 気温：...,#Qiita 投稿しました。\n\nhttps://t.co/cTLwZgbcWY\n\n#...,ブン・レモングラス チキン『ベトニャット』新宿御苑 #おは戦31102nk #おつかれ戦隊1...,PHPの質問にまだ答えられなくて悲しい💦\n知り合いでPHPのエンジニアさんとかいればよかっ...,／\n🔰未経験者必見！\n＼\n\n未経験からのIT転職は20代が有利！🚀\n転職するために...


In [109]:
# ターゲットキーワードを含むツイート文以外はNANに置換する
keyword = '#GoogleAppsScript'
for col in df_tweets_text.columns:
    flag = df_tweets_text[col].str.contains(keyword, na=False)
    df_tweets_text.loc[~flag, col] = np.nan

# NAN以外の値を持つユーザーIDを特定する（POSTする候補）
exist_flag = df_tweets_text.count() != 0
user_name = df_tweets_text.T[exist_flag].index

# POSTするユーザー候補ごとに関連ツイート文を全て結合
candidate_users = df_tweets_text[user_name].T
# 後で文を区切る区切りのためツイート文の文末に'。'を追加
candidate_users += '。'
# ツイート文を結合
candidate_users['all'] = candidate_users.iloc[:,0].str.cat([candidate_users.iloc[:,col] for col in candidate_users.columns[1:]], na_rep='')

In [120]:
can_user = candidate_users['all']
can_user

1282640602396364802    2つとも使えるようになりたい🤔🤔ww\n\n#使いたい構文 #JavaScript #Goo...
1356973982             二次元配列から特定の列だけ取り出す方法\n\nこれめちゃ便利🤔🤔\n\nconst pick...
Name: all, dtype: object

In [166]:
split_punc2 = functools.partial(split_punctuation, punctuations=r"。!?")
concat_tail_no = functools.partial(concatenate_matching, former_matching_rule=r"^(?P<result>.+)(の)$", remove_former_matched=False)
segmenter = make_pipeline(normalize, split_newline, concat_tail_no, split_punc2)

# 結合したツイート文を文単位で分割する
sentence_list = []
for text in can_user:
    sentence_list.append(list(segmenter(text)))

#can_user['text'] = df_list
can_user_sentence = pd.Series(sentence_list, index=can_user.index, name='text')

# '。'だけの文字と絵文字を削除

for i in range(len(can_user_sentence)):
    can_user_sentence[i] = [demoji.replace(string=seg, repl='') for seg in can_user_sentence[i] if '。' != seg]

In [167]:
can_user_sentence[0][:4]

['2つとも使えるようになりたいww',
 '#使いたい構文#JavaScript#GoogleAppsScript ',
 '#filter#配列',
 '#プログラミング#プログラミング初心者と繋がりたい#駆け出しエンジニアと繋がりたい#駆け出しエンジニアhttps://t.co/Ne6kOA0MVw。']

In [162]:
can_user_sentence[0]

['2つとも使えるようになりたい🤔🤔ww',
 '#使いたい構文#JavaScript#GoogleAppsScript ',
 '#filter#配列',
 '#プログラミング#プログラミング初心者と繋がりたい#駆け出しエンジニアと繋がりたい#駆け出しエンジニアhttps://t.co/Ne6kOA0MVw。']

In [168]:
vectorize_candidate_segments(can_user_sentence)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


,text,vector
1282640602396364802,"[2つとも使えるようになりたいww, #使いたい構文#JavaScript#GoogleAp...","0 [-0.18843903, -0.19478449, -0.020317644, ..."
1356973982,"[二次元配列から特定の列だけ取り出す方法, これめちゃ便利, const picked＝ht...","0 [-0.14769673, -0.20070626, -0.4178725, -0..."
